In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from random import *
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
data_train = pd.read_csv('fashion-mnist_train.csv', header = 0)

In [4]:
data_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
labels = data_train['label'].values.reshape(1, 60000) # một hàng 60000 cột
train = data_train.drop('label', axis=1).transpose()

- Nếu chỉ là data_train['label'] thì shape sẽ chỉ là (60000) tuy nhiên chúng ta cần dạng tensor nên cần reshape bằng (1,60000)
- Hàm transpose sẽ đổi hàng thành cột và cột thành hàng. Tức là như dữ liệu ban đầu (hàng là bản ghi, cột là thuộc tính, sau khi transpose thì hàng là thuộc tính, cột là bản ghi

In [5]:
train = np.array(train / 255.0)

# Bắt đầu thử với 1 hidden layer

Hình ảnh của mạng - Sách 0.3B trang 106/425

- Hàm softmax sẽ biến một vector k chiều thành 1 vector k chiều có tổng bằng 1 và các giá trị thuộc khoảng (0;1)

In [8]:
a = np.zeros((5,2))
a

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [9]:
labels.shape

(1, 60000)

Mã hóa one-hot endcoding

In [6]:
labels_ = np.zeros((60000, 10))
labels_[np.arange(60000), labels] = 1
labels_ = labels_.transpose()
labels_ = np.array(labels_)

In [11]:
labels_
labels_.shape

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

(10, 60000)

# Bắt đầu tạo model với tensorflow từ đây

In [7]:
n_dim = 784
tf.reset_default_graph()
# Number of neurons in the layers
n1 = 5 # Number of neurons in layer 1
n2 = 10 # Number of neurons in output layer
cost_history = np.empty(shape=[1], dtype = float)
learning_rate = tf.placeholder(tf.float32, shape=())
X = tf.placeholder(tf.float32, [n_dim, None])
Y = tf.placeholder(tf.float32, [10, None])
W1 = tf.Variable(tf.truncated_normal([n1, n_dim], stddev=.1))
b1 = tf.Variable(tf.zeros([n1,1]))
W2 = tf.Variable(tf.truncated_normal([n2, n1], stddev=.1))
b2 = tf.Variable(tf.zeros([n2,1]))
# Let's build our network...
Z1 = tf.nn.relu(tf.matmul(W1, X) + b1)
Z2 = tf.nn.relu(tf.matmul(W2, Z1) + b2)
y_ = tf.nn.softmax(Z2,0)
cost = - tf.reduce_mean(Y * tf.log(y_)+(1-Y) * tf.log(1-y_))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer()

- Giải thích các hàm:
- tf.truncated_normal([n1, n_dim], stddev=.1): sinh ngẫu nhiên
- tf.matmul(W1, X): tích vô hướng
- tf.reduce_mean: tính giá trị trung bình của tensor

In [42]:
l = tf.constant([[1.], [2.], [3.]])
sm = tf.nn.softmax(l, 0)
s1 = tf.Session()
s1.run(sm)

array([[0.09003057],
       [0.24472848],
       [0.66524094]], dtype=float32)

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 800
cost_history = []

In [45]:
train
labels_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [9]:
for epoch in range(training_epochs+1):
    sess.run(optimizer, feed_dict = {X: train, Y: labels_, learning_rate: 0.001})
    cost_ = sess.run(cost, feed_dict={ X:train, Y: labels_, learning_rate: 0.001})
    if cost_ is np.nan:
        print("????")
        break
    else:
        cost_history = np.append(cost_history, cost_)
        if (epoch % 20 == 0):
            print("Reached epoch",epoch,"cost J =", cost_)


Reached epoch 0 cost J = 0.32529938
Reached epoch 20 cost J = 0.30868238
Reached epoch 40 cost J = 0.28770435
Reached epoch 60 cost J = 0.26667133
Reached epoch 80 cost J = 0.24788605
Reached epoch 100 cost J = 0.23176329
Reached epoch 120 cost J = 0.21906896
Reached epoch 140 cost J = 0.21029377
Reached epoch 160 cost J = 0.20430641
Reached epoch 180 cost J = 0.19967963
Reached epoch 200 cost J = 0.19572495
Reached epoch 220 cost J = 0.19213395
Reached epoch 240 cost J = 0.18879402
Reached epoch 260 cost J = 0.18564856
Reached epoch 280 cost J = 0.1826727
Reached epoch 300 cost J = 0.17985298
Reached epoch 320 cost J = 0.17719509
Reached epoch 340 cost J = 0.1747138
Reached epoch 360 cost J = 0.17241956
Reached epoch 380 cost J = 0.17032586
Reached epoch 400 cost J = 0.1684272
Reached epoch 420 cost J = 0.16670969
Reached epoch 440 cost J = 0.16515733
Reached epoch 460 cost J = 0.16375083
Reached epoch 480 cost J = 0.16247234
Reached epoch 500 cost J = 0.16130571
Reached epoch 520 cos

In [49]:
Y

<tf.Tensor 'Placeholder_2:0' shape=(10, ?) dtype=float32>

In [10]:
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float")) # hàm cast để đổi kiểu của một tensor
print ("Accuracy:", accuracy.eval({X: train, Y: labels_, learning_rate:0.001}, session = sess))

Accuracy: 0.66643333


In [11]:
data_dev = pd.read_csv('fashion-mnist_test.csv', header = 0)
labels_dev = data_dev['label'].values.reshape(1, 10000)
labels_dev_ = np.zeros((10000, 10))
labels_dev_[np.arange(10000), labels_dev] = 1
labels_dev_ = labels_dev_.transpose()
dev = data_dev.drop('label', axis=1).transpose()

In [12]:
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
print ("Accuracy:", accuracy.eval({X: dev, Y: labels_dev_, learning_rate:0.001}, session = sess))

Accuracy: 0.6466


In [52]:
accuracy

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>

# Batch Gradient Descent

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100
cost_history = []
for epoch in range(training_epochs+1):
    sess.run(optimizer, feed_dict = {X: train, Y: labels_, learning_rate:0.01})
    cost_ = sess.run(cost, feed_dict={ X:train, Y: labels_, learning_rate:0.01})
    cost_history = np.append(cost_history, cost_)
    if (epoch % 50 == 0):
        print("Reached epoch",epoch,"cost J =", cost_)

Reached epoch 0 cost J = 0.31843516
Reached epoch 50 cost J = 0.12358896
Reached epoch 100 cost J = 0.0964246


In [23]:
correct_predictions

<tf.Tensor 'Equal:0' shape=(?,) dtype=bool>

In [14]:
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
print ("Accuracy:", accuracy.eval({X: train, Y: labels_, learning_rate:0.001}, session = sess))

Accuracy: 0.80653334


In [60]:
b = tf.equal([1.,2.,4.], [0,2.,4.])
a = tf.cast(b, "float")
acc = tf.reduce_mean(a)
print(tf.Session().run(a))
print(tf.Session().run(acc))

[0. 1. 1.]
0.6666667


# Stochastic Gradient Descent

In [63]:
train.shape
labels_.shape

(784, 60000)

(10, 60000)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
cost_history = []
for epoch in range(2+1):
    for i in range(0, train.shape[1], 1):
        X_train_mini = train[:,i:i + 1]
        y_train_mini = labels_[:,i:i + 1]
        sess.run(optimizer, feed_dict = {X: X_train_mini, Y: y_train_mini, learning_rate: 0.0001})
        cost_ = sess.run(cost, feed_dict={ X:train, Y: labels_, learning_rate: 0.0001})
    cost_history = np.append(cost_history, cost_)
    print("Reached epoch",epoch,"cost J =", cost_)
        

Reached epoch 0 cost J = 0.21477713
Reached epoch 1 cost J = 0.20312396


Chạy quá lâu

In [66]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
cost_history = []
for epoch in range(100+1):
    for i in range(0, train.shape[1], 50):
        X_train_mini = train[:,i:i + 50]
        y_train_mini = labels_[:,i:i + 50]
        sess.run(optimizer, feed_dict = {X: X_train_mini, Y: y_train_mini, learning_rate: 0.001})
        cost_ = sess.run(cost, feed_dict={ X:train, Y: labels_, learning_rate: 0.001})
    cost_history = np.append(cost_history, cost_)
    if (epoch % 50 == 0):
        print("Reached epoch",epoch,"cost J =", cost_)

Reached epoch 0 cost J = 0.20953572


KeyboardInterrupt: 

In [32]:
def model(minibatch_size, training_epochs, features, classes, logging_step = 100, learning_r = 0.001):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    cost_history = []
    for epoch in range(training_epochs+1):
        for i in range(0, features.shape[1], minibatch_size):
            X_train_mini = features[:,i:i + minibatch_size]
            y_train_mini = classes[:,i:i + minibatch_size]
            sess.run(optimizer, feed_dict = {X: X_train_mini, Y: y_train_mini, learning_rate: learning_r})
            cost_ = sess.run(cost, feed_dict={ X:features, Y: classes, learning_rate: learning_r})
            cost_history = np.append(cost_history, cost_)
        if (epoch % logging_step == 0):
            print("Reached epoch",epoch,"cost J =", cost_)
    return sess, cost_history

In [41]:
def create_layer (X, n, activation):
    ndim = int(X.shape[0])
    stddev = 2 / np.sqrt(ndim)
    initialization = tf.truncated_normal((n, ndim), stddev = stddev)
    W = tf.Variable(init)
    b = tf.Variable(tf.zeros([n,1]))
    Z = tf.matmul(W,X)+b
    return activation(Z)

In [42]:
n_dim = 784
n1 = 300
n2 = 300
n_outputs = 10
X = tf.placeholder(tf.float32, [n_dim, None])
Y = tf.placeholder(tf.float32, [10, None])
learning_rate = tf.placeholder(tf.float32, shape=())
hidden1 = create_layer (X, n1, activation = tf.nn.relu)
hidden2 = create_layer (hidden1, n2, activation = tf.nn.relu)
outputs = create_layer (hidden2, n3, activation = tf.identity)
y_ = tf.nn.softmax(outputs)
cost = - tf.reduce_mean(Y * tf.log(y_)+(1-Y) * tf.log(1-y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
def model(minibatch_size, training_epochs, features, classes, logging_step = 100, learning_r = 0.001):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    cost_history = []
    for epoch in range(training_epochs+1):
        for i in range(0, features.shape[1], minibatch_size):
            X_train_mini = features[:,i:i + minibatch_size]
            y_train_mini = classes[:,i:i + minibatch_size]
            sess.run(optimizer, feed_dict = {X: X_train_mini, Y: y_train_mini, learning_rate: learning_r})
        cost_ = sess.run(cost, feed_dict={ X:features, Y: classes, learning_rate: learning_r})
        cost_history = np.append(cost_history, cost_)
    if (epoch % logging_step == 0):
        print("Reached epoch",epoch,"cost J =", cost_)
    return sess, cost_history

TypeError: can't convert Operation 'init' to Tensor